In [103]:
import pandas as pd
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, StandardScaler
import src.util as util
import yaml
import copy
from tqdm import tqdm
import os
from datetime import datetime

In [104]:
config_data=util.config_load()

In [105]:
df_inp=util.pickle_load(config_data['raw_dataset_path_train_inp'])
df_outp=util.pickle_load(config_data['raw_dataset_path_train_outp'])
df_ben=util.pickle_load(config_data['raw_dataset_path_train_ben'])

In [106]:
df_ben

,BeneID,Gender,Race,RenalDiseaseIndicator,State,County,NoOfMonths_PartACov,NoOfMonths_PartBCov,ChronicCond_Alzheimer,ChronicCond_Heartfailure,...,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,IPAnnualReimbursementAmt,IPAnnualDeductibleAmt,OPAnnualReimbursementAmt,OPAnnualDeductibleAmt,AGE,DOD_Flag,TotalIPAnnualAmt,TotalOPAnnualAmt
0,BENE11001,1,1,0,39,230,12,12,1,0,...,1,1,36000,3204,60,70,66,0,72000,130
1,BENE11002,1,1,0,39,280,12,12,0,0,...,0,0,0,0,30,50,73,0,0,80
2,BENE11003,1,1,0,52,590,12,12,1,0,...,0,0,0,0,90,40,73,0,0,130
3,BENE11004,1,1,0,39,270,12,12,1,1,...,1,0,0,0,1810,760,87,0,0,2570
4,BENE11005,1,1,0,24,680,12,12,0,0,...,0,0,0,0,1790,1200,74,0,0,2990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138551,BENE159194,1,1,0,39,140,12,12,1,0,...,0,0,0,0,430,460,70,0,0,890
138552,BENE159195,1,1,0,49,530,12,12,1,0,...,0,0,0,0,880,100,71,0,0,980
138553,BENE159196,1,1,0,6,150,12,12,0,1,...,0,0,2000,1068,3240,1390,93,0,4000,4630
138554,BENE159197,1,1,0,16,560,12,12,1,1,...,0,0,0,0,2650,10,79,0,0,2660


In [107]:
def merge(df_inp,df_outp,df_ben):
    common_cols=[i for i in df_inp.columns if i in df_outp.columns]
    df_inp_outp=pd.merge(left=df_inp, right=df_outp, left_on=common_cols,right_on=common_cols,how='outer')
    df_inp_outp_ben=pd.merge(left=df_inp_outp,right=df_ben,left_on='BeneID',right_on='BeneID',how='left')
    return df_inp_outp_ben


In [108]:
df_inp_outp_ben=merge(df_inp,df_outp,df_ben)

In [109]:
df_inp_outp_ben['BeneID'].nunique()

138556

In [74]:
def standardize_conditions(df_ben_outp):
    for i in ['RenalDiseaseIndicator','ChronicCond_Alzheimer',
        'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
        'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
        'ChronicCond_Depression', 'ChronicCond_Diabetes',
        'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
        'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke','Gender']:
        df_ben_outp[i] = df_ben_outp[i].apply(lambda val: 0 if val =='2' else 1)
    

In [75]:
standardize_conditions(df_inp_outp_ben)

In [76]:
len(df_inp_outp_ben)

558211

**Create Interaction Terms**

#SUM OF AMOUNTS,CLAIM AND ADMIT PERIOD on PROVIDER

In [114]:
def df_group_amt_prv(df_inp_outp_ben):
    # Initialize the DataFrame with unique 'Provider' values
    df_group_data_prv_amt = pd.DataFrame(df_inp_outp_ben['Provider'].unique(), columns=['Provider'])

    # Group by 'Provider' and calculate the sum of the columns, without using 'transform'
    df_sum = df_inp_outp_ben.groupby('Provider')[['DeductibleAmtPaid', 'InscClaimAmtReimbursed', 'Admit_Period', 'Claim_period']].sum()

    # Merge the summed values with the 'df_group_data_prv_amt' DataFrame
    df_group_data_prv_amt = df_group_data_prv_amt.merge(df_sum, on='Provider', how='left')

    # Rename columns to match the desired naming convention
    df_group_data_prv_amt.rename(columns={
        'DeductibleAmtPaid': 'PRV_TotalDeductibleAmtPaid',
        'InscClaimAmtReimbursed': 'PRV_TotalInscClaimAmtReimbursed',
        'Admit_Period': 'PRV_TotalAdmitPeriod',
        'Claim_period': 'PRV_TotalClaimPeriod'
    }, inplace=True)

    return df_group_data_prv_amt
        

In [115]:
df_group_data_prv_amt=df_group_amt_prv(df_inp_outp_ben)

In [80]:
df_group_data_prv_amt

,Provider,PRV_TotalDeductibleAmtPaid,PRV_TotalInscClaimAmtReimbursed,PRV_TotalAdmitPeriod,PRV_TotalClaimPeriod
0,PRV57172,96456,1386100,516.0,2968
1,PRV55158,96456,1386100,516.0,2968
2,PRV54966,30954,508210,151.0,988
3,PRV54890,5980,502010,0.0,2287
4,PRV52145,4672,74320,35.0,193
...,...,...,...,...,...
5405,PRV54294,106092,1030600,615.0,1039
5406,PRV51053,1010,69380,0.0,412
5407,PRV55405,121008,1154530,617.0,1592
5408,PRV57061,121008,1154530,617.0,1592


In [45]:
df_group_data_prv_amt[df_group_data_prv_amt.Provider=='PRV51459']

,Provider,PRV_TotalDeductibleAmtPaid,PRV_TotalInscClaimAmtReimbursed,PRV_TotalAdmitPeriod,PRV_TotalClaimPeriod
21,PRV51459,30954,508210,151.0,988


In [81]:
df_group_data_prv_amt

,Provider,PRV_TotalDeductibleAmtPaid,PRV_TotalInscClaimAmtReimbursed,PRV_TotalAdmitPeriod,PRV_TotalClaimPeriod
0,PRV57172,96456,1386100,516.0,2968
1,PRV55158,96456,1386100,516.0,2968
2,PRV54966,30954,508210,151.0,988
3,PRV54890,5980,502010,0.0,2287
4,PRV52145,4672,74320,35.0,193
...,...,...,...,...,...
5405,PRV54294,106092,1030600,615.0,1039
5406,PRV51053,1010,69380,0.0,412
5407,PRV55405,121008,1154530,617.0,1592
5408,PRV57061,121008,1154530,617.0,1592


In [82]:
def Feature_engineering_level1(df_inp_outp_ben):
     df=pd.DataFrame()
     df_group_data_prv=pd.DataFrame(df_inp_outp_ben['Provider'].unique(),columns=['Provider'])
     for i in ['Provider']:
          for j in ['BeneID',
     'AttendingPhysician',
     'OtherPhysician',
     'OperatingPhysician',
     'ClmAdmitDiagnosisCode',
     'ClmProcedureCode_1',
     'ClmProcedureCode_2',
     'ClmProcedureCode_3',
     'ClmDiagnosisCode_1',
     'ClmDiagnosisCode_2',
     'ClmDiagnosisCode_3',
     'ClmDiagnosisCode_4',
     'ClmDiagnosisCode_5',
     'ClmDiagnosisCode_6',
     'ClmDiagnosisCode_7',
     'ClmDiagnosisCode_8',
     'ClmDiagnosisCode_9',
     'ClmDiagnosisCode_10',
     'DiagnosisGroupCode'
     ]:
               prefix=i+"_"+j
               df=df_inp_outp_ben.groupby([i,j])['ClaimID'].count().reset_index()
               df=df.rename(columns={'ClaimID':'Clm_cnt_'+prefix})
               df=df.groupby(['Provider'])['Clm_cnt_'+prefix].sum().reset_index()
               df_group_data_prv=df_group_data_prv.merge(df,on='Provider',how='left')
     return df_group_data_prv


In [83]:
def Feature_engineering_level3_phy(df_inp_outp_ben):
     df=pd.DataFrame()
     df_group_data_prv_ben_phy=pd.DataFrame(df_inp_outp_ben['Provider'].unique(),columns=['Provider'])
     for i in ['Provider']:
          for j in ['BeneID']:
               for k in [
          'AttendingPhysician',
          'OtherPhysician',
          'OperatingPhysician']:
                    for l in ['ClmAdmitDiagnosisCode',
          'ClmProcedureCode_1',
          'ClmProcedureCode_2',
          'ClmProcedureCode_3',
          'ClmDiagnosisCode_1',
          'ClmDiagnosisCode_2',
          'ClmDiagnosisCode_3',
          'ClmDiagnosisCode_4',
          'ClmDiagnosisCode_5',
          'ClmDiagnosisCode_6',
          'ClmDiagnosisCode_7',
          'ClmDiagnosisCode_8',
          'ClmDiagnosisCode_9',
          'ClmDiagnosisCode_10',
          'DiagnosisGroupCode']:
                         prefix=i+"_"+j+"_"+k+"_"+l
                         df=df_inp_outp_ben.groupby([i,j,k,l])['ClaimID'].count().reset_index()
                         df=df.rename(columns={'ClaimID':'Clm_cnt_'+prefix})
                         df=df.groupby(['Provider'])['Clm_cnt_'+prefix].sum().reset_index()
                         df_group_data_prv_ben_phy=df_group_data_prv_ben_phy.merge(df,on='Provider',how='left')
     return df_group_data_prv_ben_phy

In [85]:
def Feature_engineering_level3_diag(df_inp_outp_ben):
     df=pd.DataFrame()
     df_group_data_prv_ben_diag=pd.DataFrame(df_inp_outp_ben['Provider'].unique(),columns=['Provider'])
     for i in ['Provider']:
          for j in ['BeneID']:
               for k in [
          'ClmDiagnosisCode_1',
          'ClmDiagnosisCode_2',
          'ClmDiagnosisCode_3',
          'ClmDiagnosisCode_4',
          'ClmDiagnosisCode_5',
          'ClmDiagnosisCode_6',
          'ClmDiagnosisCode_7',
          'ClmDiagnosisCode_8',
          'ClmDiagnosisCode_9',
          'ClmDiagnosisCode_10',]:
                    for l in [
          'ClmProcedureCode_1',
          'ClmProcedureCode_2',
          'ClmProcedureCode_3',
          ]:
                         prefix=i+"_"+j+"_"+k+"_"+l
                         df=df_inp_outp_ben.groupby([i,j,k,l])['ClaimID'].count().reset_index()
                         df=df.rename(columns={'ClaimID':'Clm_cnt_'+prefix})
                         df=df.groupby(['Provider'])['Clm_cnt_'+prefix].sum().reset_index()
                         df_group_data_prv_ben_diag=df_group_data_prv_ben_diag.merge(df,on='Provider',how='left')
     return df_group_data_prv_ben_diag

In [86]:
df_grp_data_prv=Feature_engineering_level1(df_inp_outp_ben)
df_grp_data_prv_ben_phy=Feature_engineering_level3_phy(df_inp_outp_ben)
df_grp_data_prv_ben_diag=Feature_engineering_level3_diag(df_inp_outp_ben)

In [87]:
df_grp_data_prv

,Provider,Clm_cnt_Provider_BeneID,Clm_cnt_Provider_AttendingPhysician,Clm_cnt_Provider_OtherPhysician,Clm_cnt_Provider_OperatingPhysician,Clm_cnt_Provider_ClmAdmitDiagnosisCode,Clm_cnt_Provider_ClmProcedureCode_1,Clm_cnt_Provider_ClmProcedureCode_2,Clm_cnt_Provider_ClmProcedureCode_3,Clm_cnt_Provider_ClmDiagnosisCode_1,Clm_cnt_Provider_ClmDiagnosisCode_2,Clm_cnt_Provider_ClmDiagnosisCode_3,Clm_cnt_Provider_ClmDiagnosisCode_4,Clm_cnt_Provider_ClmDiagnosisCode_5,Clm_cnt_Provider_ClmDiagnosisCode_6,Clm_cnt_Provider_ClmDiagnosisCode_7,Clm_cnt_Provider_ClmDiagnosisCode_8,Clm_cnt_Provider_ClmDiagnosisCode_9,Clm_cnt_Provider_ClmDiagnosisCode_10,Clm_cnt_Provider_DiagnosisGroupCode
0,PRV57172,1760,1755,662.0,340.0,447.0,50.0,13.0,NaN,1724.0,1155.0,755.0,502.0,332.0,253.0,183.0,147.0,121.0,14.0,88.0
1,PRV55158,698,696,242.0,129.0,172.0,17.0,3.0,NaN,690.0,452.0,305.0,214.0,127.0,93.0,66.0,53.0,47.0,2.0,29.0
2,PRV54966,1796,1792,690.0,283.0,390.0,1.0,NaN,NaN,1754.0,1106.0,694.0,434.0,248.0,162.0,104.0,79.0,59.0,5.0,NaN
3,PRV54890,77,76,31.0,12.0,22.0,2.0,1.0,NaN,77.0,54.0,33.0,23.0,18.0,15.0,11.0,8.0,7.0,1.0,4.0
4,PRV52145,245,244,96.0,40.0,47.0,NaN,NaN,NaN,237.0,164.0,102.0,61.0,33.0,23.0,14.0,7.0,4.0,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV54294,1,1,NaN,1.0,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN
5406,PRV51053,1,1,1.0,1.0,1.0,1.0,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0
5407,PRV55405,10,10,7.0,4.0,NaN,NaN,NaN,NaN,10.0,7.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
5408,PRV57061,1,1,NaN,NaN,1.0,NaN,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
df_outp[df_outp.Provider=='PRV51459']

,BeneID,ClaimID,Provider,InscClaimAmtReimbursed,AttendingPhysician,OperatingPhysician,OtherPhysician,ClmDiagnosisCode_1,ClmDiagnosisCode_2,ClmDiagnosisCode_3,...,ClmProcedureCode_1,ClmProcedureCode_2,ClmProcedureCode_3,DeductibleAmtPaid,ClmAdmitDiagnosisCode,Claim_period,Beneficiary_cost,Count_diag_code,Count_proc_code,Is_admit
96,BENE11023,CLM516020,PRV51459,60,PHY357120,NaN,NaN,4414,NaN,NaN,...,NaN,NaN,NaN,0,4414,0,60,1,0,0
97,BENE11023,CLM591356,PRV51459,100,PHY338032,NaN,PHY338032,2720,NaN,NaN,...,NaN,NaN,NaN,0,NaN,0,100,1,0,0
98,BENE11023,CLM613474,PRV51459,10,PHY327046,NaN,PHY341578,4011,2724,78843,...,NaN,NaN,NaN,0,NaN,0,10,3,0,0
99,BENE11023,CLM740365,PRV51459,40,PHY327046,NaN,PHY341578,2722,2749,NaN,...,NaN,NaN,NaN,0,NaN,0,40,2,0,0
100,BENE11023,CLM744555,PRV51459,70,PHY314027,NaN,PHY337425,4019,78079,7851,...,NaN,NaN,NaN,0,NaN,0,70,4,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517265,BENE159078,CLM714286,PRV51459,30,PHY341578,NaN,NaN,5600,7580,78605,...,NaN,NaN,NaN,0,NaN,3,30,7,0,0
517460,BENE159125,CLM697465,PRV51459,50,PHY341578,PHY341578,PHY341578,2940,2761,7812,...,NaN,NaN,NaN,0,3310,0,50,3,0,0
517598,BENE159166,CLM345904,PRV51459,60,PHY327046,NaN,PHY341578,4660,NaN,NaN,...,NaN,NaN,NaN,100,NaN,0,-40,1,0,0
517610,BENE159169,CLM201892,PRV51459,50,PHY423534,NaN,NaN,3320,NaN,NaN,...,NaN,NaN,NaN,0,NaN,0,50,1,0,0


unique counts of DRG codes, diagnosis admit code

In [89]:
def Feature_engineering_unique_data(df_inp_outp_ben):
     df=pd.DataFrame()
     df_group_data_prv_unq=pd.DataFrame(df_inp_outp_ben['Provider'].unique(),columns=['Provider'])
     for i in ['DiagnosisGroupCode','ClmAdmitDiagnosisCode']:
            
                         df=df_inp_outp_ben.groupby(['Provider'])[i].nunique().reset_index()
                         df=df.rename(columns={i:'Prv_unq_cnt_'+i})
                         df=df.groupby(['Provider'])['Prv_unq_cnt_'+i].sum().reset_index()
                         df_group_data_prv_unq=df_group_data_prv_unq.merge(df,on='Provider',how='left')
     return df_group_data_prv_unq

In [90]:
df_grp_data_prv_unique=Feature_engineering_unique_data(df_inp_outp_ben)

In [91]:
df_grp_data_prv_unique

,Provider,Prv_unq_cnt_DiagnosisGroupCode,Prv_unq_cnt_ClmAdmitDiagnosisCode
0,PRV57172,76,253
1,PRV55158,29,129
2,PRV54966,0,226
3,PRV54890,4,20
4,PRV52145,0,41
...,...,...,...
5405,PRV54294,0,0
5406,PRV51053,1,1
5407,PRV55405,0,0
5408,PRV57061,0,1


Grouping Existing dataframe on provider level

Drop unwanted fields

In [93]:
df_inp_outp_ben.drop(columns=['BeneID', 'ClaimID','InscClaimAmtReimbursed',
       'AttendingPhysician', 'OperatingPhysician', 'OtherPhysician', 'DeductibleAmtPaid','Claim_period', 'Beneficiary_cost',
       'Count_diag_code', 'Count_proc_code', 'Admit_Period','Race','State', 'County','IPAnnualReimbursementAmt',
       'IPAnnualDeductibleAmt', 'OPAnnualReimbursementAmt',
       'OPAnnualDeductibleAmt', 'AGE','TotalIPAnnualAmt',
       'TotalOPAnnualAmt'],inplace=True)

In [94]:
df_column_grouping=df_inp_outp_ben[['Provider','Is_admit', 'Gender', 'RenalDiseaseIndicator', 'ChronicCond_Alzheimer',
       'ChronicCond_Heartfailure', 'ChronicCond_KidneyDisease',
       'ChronicCond_Cancer', 'ChronicCond_ObstrPulmonary',
       'ChronicCond_Depression', 'ChronicCond_Diabetes',
       'ChronicCond_IschemicHeart', 'ChronicCond_Osteoporasis',
       'ChronicCond_rheumatoidarthritis', 'ChronicCond_stroke', 'DOD_Flag']]

In [95]:
df_inp_outp_ben_grp=df_column_grouping.groupby('Provider').sum()

In [96]:
df_inp_outp_ben_grp

,Is_admit,Gender,RenalDiseaseIndicator,ChronicCond_Alzheimer,ChronicCond_Heartfailure,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,DOD_Flag
Provider,,,,,,,,,,,,,,,
PRV51001,5,25,25,25,25,25,25,25,25,25,25,25,25,25,0
PRV51003,62,132,132,132,132,132,132,132,132,132,132,132,132,132,1
PRV51004,0,149,149,149,149,149,149,149,149,149,149,149,149,149,1
PRV51005,0,1165,1165,1165,1165,1165,1165,1165,1165,1165,1165,1165,1165,1165,4
PRV51007,3,72,72,72,72,72,72,72,72,72,72,72,72,72,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PRV57759,0,28,28,28,28,28,28,28,28,28,28,28,28,28,0
PRV57760,0,22,22,22,22,22,22,22,22,22,22,22,22,22,0
PRV57761,0,82,82,82,82,82,82,82,82,82,82,82,82,82,1


In [97]:
data_combined=df_grp_data_prv.merge(df_grp_data_prv_ben_phy,on='Provider').merge(df_grp_data_prv_ben_diag,on='Provider').merge(df_group_data_prv_amt,on='Provider').merge(df_grp_data_prv_unique,on='Provider').merge(df_inp_outp_ben_grp,on='Provider')

In [98]:
data_combined

,Provider,Clm_cnt_Provider_BeneID,Clm_cnt_Provider_AttendingPhysician,Clm_cnt_Provider_OtherPhysician,Clm_cnt_Provider_OperatingPhysician,Clm_cnt_Provider_ClmAdmitDiagnosisCode,Clm_cnt_Provider_ClmProcedureCode_1,Clm_cnt_Provider_ClmProcedureCode_2,Clm_cnt_Provider_ClmProcedureCode_3,Clm_cnt_Provider_ClmDiagnosisCode_1,...,ChronicCond_KidneyDisease,ChronicCond_Cancer,ChronicCond_ObstrPulmonary,ChronicCond_Depression,ChronicCond_Diabetes,ChronicCond_IschemicHeart,ChronicCond_Osteoporasis,ChronicCond_rheumatoidarthritis,ChronicCond_stroke,DOD_Flag
0,PRV57172,1760,1755,662.0,340.0,447.0,50.0,13.0,NaN,1724.0,...,1760,1760,1760,1760,1760,1760,1760,1760,1760,3
1,PRV55158,698,696,242.0,129.0,172.0,17.0,3.0,NaN,690.0,...,698,698,698,698,698,698,698,698,698,7
2,PRV54966,1796,1792,690.0,283.0,390.0,1.0,NaN,NaN,1754.0,...,1796,1796,1796,1796,1796,1796,1796,1796,1796,15
3,PRV54890,77,76,31.0,12.0,22.0,2.0,1.0,NaN,77.0,...,77,77,77,77,77,77,77,77,77,1
4,PRV52145,245,244,96.0,40.0,47.0,NaN,NaN,NaN,237.0,...,245,245,245,245,245,245,245,245,245,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5405,PRV54294,1,1,NaN,1.0,NaN,NaN,NaN,NaN,1.0,...,1,1,1,1,1,1,1,1,1,0
5406,PRV51053,1,1,1.0,1.0,1.0,1.0,NaN,NaN,1.0,...,1,1,1,1,1,1,1,1,1,0
5407,PRV55405,10,10,7.0,4.0,NaN,NaN,NaN,NaN,10.0,...,10,10,10,10,10,10,10,10,10,0
5408,PRV57061,1,1,NaN,NaN,1.0,NaN,NaN,NaN,1.0,...,1,1,1,1,1,1,1,1,1,0


In [99]:
data_combined.fillna(0,inplace=True)

In [110]:
df_inp_outp_ben_simil_data=df_inp_outp_ben[['Provider','BeneID','Is_admit','ClmAdmitDiagnosisCode','DiagnosisGroupCode', 'ClmDiagnosisCode_1',
       'ClmDiagnosisCode_2', 'ClmDiagnosisCode_3', 'ClmDiagnosisCode_4',
       'ClmDiagnosisCode_5', 'ClmDiagnosisCode_6', 'ClmDiagnosisCode_7',
       'ClmDiagnosisCode_8', 'ClmDiagnosisCode_9', 'ClmDiagnosisCode_10',
       'ClmProcedureCode_1', 'ClmProcedureCode_2', 'ClmProcedureCode_3']]

#Pickling

In [111]:
util.pickle_dump(data_combined, config_data["raw_dataset_path_data_combined"])
util.pickle_dump(df_inp_outp_ben_simil_data, config_data["raw_dataset_path_data_simil"])
